You are given a 0-indexed integer array nums of length n. You are initially standing at index 0. You can jump from index i to index j where i < j if:

nums[i] <= nums[j] and nums[k] < nums[i] for all indexes k in the range i < k < j, or
nums[i] > nums[j] and nums[k] >= nums[i] for all indexes k in the range i < k < j.
You are also given an integer array costs of length n where costs[i] denotes the cost of jumping to index i.

Return the minimum cost to jump to the index n - 1.



Example 1:

Input: nums = [3,2,4,4,1], costs = [3,7,6,4,2]
Output: 8
Explanation: You start at index 0.
- Jump to index 2 with a cost of costs[2] = 6.
- Jump to index 4 with a cost of costs[4] = 2.
The total cost is 8. It can be proven that 8 is the minimum cost needed.
Two other possible paths are from index 0 -> 1 -> 4 and index 0 -> 2 -> 3 -> 4.
These have a total cost of 9 and 12, respectively.
Example 2:

Input: nums = [0,1,2], costs = [1,1,1]
Output: 2
Explanation: Start at index 0.
- Jump to index 1 with a cost of costs[1] = 1.
- Jump to index 2 with a cost of costs[2] = 1.
The total cost is 2. Note that you cannot jump directly from index 0 to index 2 because nums[0] <= nums[1].


Constraints:

* n == nums.length == costs.length
* 1 <= n <= 105
* 0 <= nums[i], costs[i] <= 105

# Monotonic Stacks

Let us analyze condition 1) which states that nums[i] <= nums[j] and nums[k] < nums[i] for k in range i < k < j

We will create a strictly monotonically decreasing stack to hold elements to be processed. If an element fulfills the conditions nums[j-1] <= nums[j], we will pop it. Since the stack will be strictly monotonically decreasing, suppose the next element in the stack is j-2, then if nums[j-2] <= nums[j], we will pop it as well. As the stack is strictly monotonically decreasing, it must have been that nums[j-2] > nums[j-1], so fulfilling the second part of the condition is a given.

When an element gets popped, e.g. we pop j-2 when the current index is j, then we say that element j-2 points to element j.

In the two following examples, `jump_idx` is an array denoting that the i'th idx in the array can jump to idx `jump_idx[i]` as the conditions for 1) or 2) have been fulfilled.

In [11]:
nums = [3,2,4,4,1]
n = len(nums)
mono_decreasing = []
jump_idx = [n] * n

for j in range(n):
    while len(mono_decreasing) > 0 and nums[mono_decreasing[-1]] <= nums[j]:
        jump_idx[mono_decreasing.pop()] = j

    mono_decreasing.append(j)

print(jump_idx)

[2, 2, 3, 5, 5]


Let us analyze condition 2) which states that nums[i] > nums[j] and nums[k] >= nums[i] for k in range i < k < j

We will create a monotonically increasing stack to hold elements to be processed. If an element fulfills the conditions nums[j-1] > nums[j], we will pop it. Since the stack will be monotonically increasing, suppose the next element in the stack is j-2, then if nums[j-2] > nums[j], we will pop it as well. As the stack is monotonically increasing, it must have been that nums[j-2] <= nums[j-1], so fulfilling the second part of the condition is a given.

When an element gets popped, e.g. we pop j-2 when the current index is j, then we say that element j-2 points to element j.

In [12]:
nums = [3,2,4,4,1]
n = len(nums)
mono_increasing = []
jump_idx = [n] * n

for j in range(n):
    while len(mono_increasing) > 0 and nums[mono_increasing[-1]] > nums[j]:
        jump_idx[mono_increasing.pop()] = j

    mono_increasing.append(j)

print(jump_idx)

[1, 4, 4, 4, 5]


# Dynamic Programming

Once we know which positions each element can jump to, we can employ dynamic programming to find the minimum cost to reach the end of the array.

For the range $i\in[n]$, we say that the cost of reaching element $j$ is $tab[i] + cost[j]$ if $tab[j]$ was not filled before. $tab[i]$ is the current minimal cost of reaching element $i$. If $tab[j]$ has been filled before, we must compare our best answers.

At the end, the minimal cost will be in $tab[n-1]$ which represents the minimal cost to reach the last element.

In [13]:
# imports
from typing import *

In [14]:
class Solution:

    def getCanJump(self, nums: List[int]) -> Tuple[List[int], List[int]]:
        n = len(nums)
        mono_decreasing = []
        mono_increasing = []
        jump_idx_one = [n] * n
        jump_idx_two = [n] * n
        for j in range(n):
            while len(mono_decreasing) > 0 and nums[mono_decreasing[-1]] <= nums[j]:
                jump_idx_one[mono_decreasing.pop()] = j
            while len(mono_increasing) > 0 and nums[mono_increasing[-1]] > nums[j]:
                jump_idx_two[mono_increasing.pop()] = j

            mono_decreasing.append(j)
            mono_increasing.append(j)

        return jump_idx_one, jump_idx_two

    def minCost(self, nums: List[int], costs: List[int], verbose:bool=False) -> int:
        n = len(nums)
        if n == 1:
            # base case
            return 0

        jump_idx_one, jump_idx_two = self.getCanJump(nums)

        tab = [float('inf') for _ in range(n)]
        tab[0] = costs[0] = 0  # we start at idx=0, so no cost for reaching this state

        for i in range(n):
            for j in (jump_idx_one[i], jump_idx_two[i]):
                if j < n:
                    # either element j already has the minimum cost of reaching the end,
                    # or the element j achieves a minimal cost when the cost of reaching
                    # element j from element i is accrued plus the minimal cost of
                    # reaching element i
                    tab[j] = min(tab[j], costs[j] + tab[i])

        if verbose:
            print(f"jump_idx_one:\n{jump_idx_one}")
            print(f"jump_idx_two:\n{jump_idx_two}")
            print(f"tab:\n{tab}")

        return tab[-1]

def main():
    test_cases = {
        "1": {
            "nums": [3,2,4,4,1],
            "costs": [3,7,6,4,2],
            "expected": 8
        },
        "2": {
            "nums": [5,0,2,2,1],
            "costs": [1,2,4,4,0],
            "expected": 6
        },
        "3": {
            "nums": [2,4,2,2],
            "costs": [0,2,1,5],
            "expected": 8
        }
    }

    solution = Solution()

    for tk, targs in test_cases.items():
        expected = targs.pop("expected", None)
        ret = solution.minCost(**targs, verbose=True)
        if expected is not None:
            passed = expected == ret
        else:
            passed = None
        print(f"test case {tk}: {targs}\nReturned: {ret}, Expected: {expected}\nPassed:{passed}")

main()

jump_idx_one:
[2, 2, 3, 5, 5]
jump_idx_two:
[1, 4, 4, 4, 5]
tab:
[0, 7, 6, 10, 8]
test case 1: {'nums': [3, 2, 4, 4, 1], 'costs': [0, 7, 6, 4, 2]}
Returned: 8, Expected: 8
Passed:True
jump_idx_one:
[5, 2, 3, 5, 5]
jump_idx_two:
[1, 5, 4, 4, 5]
tab:
[0, 2, 6, 10, 6]
test case 2: {'nums': [5, 0, 2, 2, 1], 'costs': [0, 2, 4, 4, 0]}
Returned: 6, Expected: 6
Passed:True
jump_idx_one:
[1, 4, 3, 4]
jump_idx_two:
[4, 2, 4, 4]
tab:
[0, 2, 3, 8]
test case 3: {'nums': [2, 4, 2, 2], 'costs': [0, 2, 1, 5]}
Returned: 8, Expected: 8
Passed:True
